# Building Baseline

We are deciding to start with KNN model as it trains quickly, and easily outputs multiple options

In [2]:
import sys
sys.path.append("/user/antonis/code/Ant-mel/legendary_game_recs/")

import pandas as pd
from sklearn.neighbors import KNeighborsRegressor

from sklearn.preprocessing import MultiLabelBinarizer, PowerTransformer
from sklearn.compose import ColumnTransformer


In [26]:
final_data = pd.read_json('../raw_data/final_data')


In [4]:
final_data.columns


Index(['title', 'release_date', 'plays', 'playing', 'backlogs', 'wishlist',
       'developers', 'avg_review', 'genres', 'platforms', 'description',
       'total_reviews', 'total_lists', 'main', 'ratings_zero_five',
       'ratings_one_zero', 'ratings_one_five', 'ratings_two_zero',
       'ratings_two_five', 'ratings_three_zero', 'ratings_three_five',
       'ratings_four_zero', 'ratings_four_five', 'ratings_five_zero', 'url',
       'game_id'],
      dtype='object')

In [5]:
# Columns we don't need immediately
cols_to_drop = ['title', 'release_date', 'plays', 'playing', 'backlogs', 'wishlist',
       'developers', 'platforms', 'description',
       'total_reviews', 'main', 'ratings_zero_five',
       'ratings_one_zero', 'ratings_one_five', 'ratings_two_zero',
       'ratings_two_five', 'ratings_three_zero', 'ratings_three_five',
       'ratings_four_zero', 'ratings_four_five', 'ratings_five_zero', 'url',
       'game_id']


In [23]:
final_useful_columns = final_data.drop(columns=cols_to_drop)
final_useful_columns


,avg_review,total_lists
0,0.0,0
1,0.0,1
2,0.0,5
3,0.0,0
4,0.0,36
...,...,...
37910,3.2,10
37911,2.9,9
37912,3.5,16
37913,4.0,0


In [7]:
# Initilizing encoder and transforming the data
mlb_genre = MultiLabelBinarizer()
transformed_genre = mlb_genre.fit_transform(final_useful_columns['genres'])

# Labeling newly created features
genre_ohe_colums = pd.DataFrame(transformed_genre, columns=mlb_genre.classes_)


In [8]:
# Concatinating the data
final_useful_columns.drop('genres', axis=1, inplace=True)
baseline_data = pd.concat((final_useful_columns, genre_ohe_colums), axis=1)


In [9]:
# Creating baseline
X_train_baseline = baseline_data.drop('avg_review', axis=1)
y_train_baseline = baseline_data['avg_review']

baseline_knn = KNeighborsRegressor(n_neighbors=10).fit(X_train_baseline,
                                                       y_train_baseline)


In [10]:
# Used to get game index to test model
final_data[(final_data['title'] == 'Call of Duty: Black Ops 4')
               | (final_data['title'] == 'The Legend of Zelda: Breath of the Wild')
              | (final_data['title'] == 'Omori')
              | (final_data['title'] == 'FIFA 21')
              | (final_data['title'] == 'Tetris')
              | (final_data['title'] == 'Sid Meier\'s Civilization V')
              | (final_data['title'] == 'Red Dead Redemption')
              | (final_data['title'] == 'Grand Theft Auto V')
                 | (final_data['title'] == 'Pikmin 4')
              | (final_data['title'] == 'The Sims 4')
              | (final_data['title'] == 'Animal Crossing: New Horizons')]


,title,release_date,plays,playing,backlogs,wishlist,developers,avg_review,genres,platforms,...,ratings_one_five,ratings_two_zero,ratings_two_five,ratings_three_zero,ratings_three_five,ratings_four_zero,ratings_four_five,ratings_five_zero,url,game_id
841,Red Dead Redemption,1692230400000,40,6,14,35,"[Double Eleven, Rockstar Games]",4.1,"[Adventure, RPG, Shooter]","[PlayStation 4, Nintendo Switch]",...,0,0,0,0,3,2,2,1,/games/red-dead-redemption--2/,260737
906,Pikmin 4,1689897600000,2100,573,1400,2300,[Nintendo],4.3,"[Adventure, Real Time Strategy, Strategy]",[Nintendo Switch],...,5,11,15,52,114,373,520,577,/games/pikmin-4/,59843
2583,Grand Theft Auto V,1655251200000,53,3,8,2,[],4.3,[],"[PlayStation 4, Xbox One, PlayStation 5, Xbox ...",...,0,0,0,1,1,9,9,8,/games/grand-theft-auto-v--2/,239064
5919,FIFA 21,1602115200000,1600,39,57,20,"[Electronic Arts, EA Vancouver]",2.5,"[Simulator, Sport]","[Windows PC, PlayStation 4, Xbox One, PlayStat...",...,90,176,211,255,114,70,18,12,/games/fifa-21/,134101
6988,Animal Crossing: New Horizons,1584576000000,27000,2100,1900,1500,"[Nintendo EPD, Nintendo]",3.6,[Simulator],[Nintendo Switch],...,189,629,1141,2833,3691,4224,1761,1417,/games/animal-crossing-new-horizons/,109462
12552,The Legend of Zelda: Breath of the Wild,1488499200000,44000,3500,6900,3600,"[Nintendo EPD Production Group No. 3, Nintendo]",4.4,"[Adventure, Puzzle, RPG]","[Wii U, Nintendo Switch]",...,131,367,563,1257,2163,5236,6908,13704,/games/the-legend-of-zelda-breath-of-the-wild/,7346
16815,The Sims 4,1409616000000,13000,666,1100,139,"[Electronic Arts, The Sims Studio]",3.1,"[RPG, Simulator]","[Windows PC, Mac, PlayStation 4, Xbox One, Pla...",...,202,617,952,1844,1355,938,206,224,/games/the-sims-4/,3212
21323,Sid Meier's Civilization V,1285027200000,7300,70,506,179,"[2K Games, Firaxis Games]",4.0,"[Simulator, Strategy, Turn Based Strategy]","[Windows PC, Mac, Linux]",...,20,50,102,376,683,1485,754,570,/games/sid-meier-s-civilization-v/,866
36255,Omori,1608854400000,11000,1400,6700,5200,"[OMOCAT, PLAYISM]",4.1,"[Adventure, Indie, RPG, Turn Based Strategy]","[Windows PC, Mac, PlayStation 4, Nintendo 3DS,...",...,104,191,254,491,766,1552,1587,3009,/games/omori/,26673
36689,Call of Duty: Black Ops 4,1539302400000,5100,22,414,145,"[Treyarch, Activision]",2.3,[Shooter],"[Windows PC, PlayStation 4, Xbox One]",...,360,598,554,607,296,176,45,23,/games/call-of-duty-black-ops-4/,83727


In [11]:
# Function that shows recomendations
def predict_baseline_model(indicies, model, reference_data, X_train):
    game = X_train[indicies:indicies+1]

    ind_list = list(model.kneighbors(game,n_neighbors=10)[1][0])
    prediction = reference_data.iloc[ind_list]

    return prediction


# Evaluation
## Breath of the wild

In [12]:
predict_baseline_model(12552, baseline_knn, final_data, X_train_baseline)


,title,release_date,plays,playing,backlogs,wishlist,developers,avg_review,genres,platforms,...,ratings_one_five,ratings_two_zero,ratings_two_five,ratings_three_zero,ratings_three_five,ratings_four_zero,ratings_four_five,ratings_five_zero,url,game_id
12552,The Legend of Zelda: Breath of the Wild,1488499200000,44000,3500,6900,3600,"[Nintendo EPD Production Group No. 3, Nintendo]",4.4,"[Adventure, Puzzle, RPG]","[Wii U, Nintendo Switch]",...,131,367,563,1257,2163,5236,6908,13704,/games/the-legend-of-zelda-breath-of-the-wild/,7346
3112,Elden Ring,1645747200000,28000,4800,7600,6600,"[FromSoftware, Bandai Namco Entertainment]",4.5,"[Adventure, RPG]","[Windows PC, PlayStation 4, Xbox One, PlayStat...",...,66,193,270,703,1207,3133,5152,10720,/games/elden-ring/,119133
15176,Undertale,1442275200000,39000,907,7400,2600,"[tobyfox, 8-4]",4.3,"[Adventure, Indie, RPG, Turn Based Strategy]","[Windows PC, Mac, Linux, PlayStation 4, Xbox O...",...,175,400,540,1492,2521,5560,5488,9709,/games/undertale/,12517
15954,Bloodborne,1427155200000,24000,1500,8300,4800,"[FromSoftware, Sony Computer Entertainment]",4.5,"[Adventure, RPG]",[PlayStation 4],...,67,186,207,492,851,2304,3446,9048,/games/bloodborne/,7334
9546,Red Dead Redemption 2,1540512000000,30000,2700,8800,4300,"[Take-Two Interactive, Rockstar Games]",4.5,"[Adventure, RPG, Shooter]","[Windows PC, PlayStation 4, Xbox One, Google S...",...,138,335,377,787,1227,2745,4002,11155,/games/red-dead-redemption-2/,25076
20664,Portal 2,1303084800000,42000,753,6400,1700,"[Electronic Arts, Valve]",4.4,"[Adventure, Platform, Puzzle, Shooter]","[Windows PC, Mac, Xbox 360, Linux, PlayStation...",...,32,97,201,790,1913,5783,6518,11007,/games/portal-2/,72
12597,Hollow Knight,1487894400000,31000,3300,12000,3300,[Team Cherry],4.4,"[Adventure, Indie, Platform]","[Windows PC, Mac, Wii U, Linux, Nintendo Switch]",...,108,302,402,914,1413,3357,4417,9780,/games/hollow-knight/,14593
10483,God of War,1524182400000,32000,1400,7100,3600,"[Sony Interactive Entertainment, SIE Santa Mon...",4.2,"[Adventure, Brawler, RPG]","[Windows PC, PlayStation 4]",...,111,333,464,1130,2098,5179,5582,6907,/games/god-of-war--1/,19560
11363,Super Mario Odyssey,1509062400000,35000,1000,4400,2800,[Nintendo],4.2,"[Adventure, Platform]",[Nintendo Switch],...,73,219,393,1331,2480,5862,6124,6483,/games/super-mario-odyssey/,26758
10917,Celeste,1516838400000,29000,1800,8500,2800,"[Maddy Makes Games, Extremely OK Games]",4.3,"[Adventure, Indie, Platform]","[Windows PC, Mac, Linux, PlayStation 4, Xbox O...",...,68,246,369,993,2049,4728,4598,6399,/games/celeste/,26226


Undertale, Hollow Knight and Celeste are closer to platfromer than Breath of the wild
Elden Ring, although equally excellent, is a totally different vibe to Breath of the Wild
4/9 are not good recommendations, and the closest one is not good enough

## Fifa 21


In [13]:
predict_baseline_model(5919, baseline_knn, final_data, X_train_baseline)


,title,release_date,plays,playing,backlogs,wishlist,developers,avg_review,genres,platforms,...,ratings_one_five,ratings_two_zero,ratings_two_five,ratings_three_zero,ratings_three_five,ratings_four_zero,ratings_four_five,ratings_five_zero,url,game_id
5919,FIFA 21,1602115200000,1600,39,57,20,"[Electronic Arts, EA Vancouver]",2.5,"[Simulator, Sport]","[Windows PC, PlayStation 4, Xbox One, PlayStat...",...,90,176,211,255,114,70,18,12,/games/fifa-21/,134101
7903,FIFA 20,1569542400000,2100,12,69,11,"[EA Canada, Electronic Arts]",2.4,[Sport],"[Windows PC, PlayStation 4, Xbox One]",...,135,242,234,262,146,68,11,21,/games/fifa-20/,114287
20254,Ico & Shadow of the Colossus Collection,1316649600000,487,3,182,43,"[Team Ico, Sony Computer Entertainment]",4.4,"[Adventure, Simulator]",[PlayStation 3],...,0,0,4,4,20,42,51,91,/games/ico-and-shadow-of-the-colossus-collection/,21084
16444,Crossy Road,1416441600000,5100,7,70,16,[Hipster Whale],2.5,[Arcade],"[Android, iOS, Web browser, Windows Phone, Ama...",...,132,413,473,460,175,95,21,47,/games/crossy-road/,10635
20602,Brink,1304985600000,1600,2,161,38,"[Splash Damage, Bethesda Softworks]",1.8,[Shooter],"[Windows PC, Xbox 360, PlayStation 3]",...,147,195,114,83,51,11,4,9,/games/brink/,502
31819,Elevator Action Returns,820368000000,178,2,121,87,[Taito],4.0,[Shooter],"[Arcade, Sega Saturn]",...,1,2,3,10,15,61,17,19,/games/elevator-action-returns/,11729
31127,Dragon Ball GT: Final Bout,872121600000,1500,2,45,36,"[Tose, Bandai]",1.8,[Fighting],[PlayStation],...,122,146,86,84,32,30,8,14,/games/dragon-ball-gt-final-bout/,2558
24414,Endless Ocean,1186012800000,340,4,141,73,"[Nintendo, Arika Co., Ltd.]",3.5,"[Adventure, Simulator]",[Wii],...,3,10,5,37,47,47,17,12,/games/endless-ocean/,4832
21663,Trauma Team,1272585600000,283,13,260,193,[Atlus],4.0,"[Adventure, Point-and-Click, Simulator]","[Wii U, Wii]",...,0,2,7,18,26,48,39,23,/games/trauma-team/,1527
25008,Tony Hawk's Project 8,1162857600000,1200,2,110,50,"[Activision, Shaba Games]",3.0,"[Racing, Sport]","[Xbox, Xbox 360, PlayStation 3, PlayStation 2,...",...,29,74,110,177,131,73,16,22,/games/tony-hawk-s-project-8/,6204


First recomendation makes sense, but the only other sport game comes in at option 9...
Terrible.

## Scaling the Data for New Model

Choosing Yeo-Johnson Scaling as it handles 0 values

In [14]:
num_transformer_yeo = PowerTransformer(method='yeo-johnson', standardize=False)

col_transformer = ColumnTransformer([('num_transformer', num_transformer_yeo,
                                   ['total_lists'])],
                                  remainder='passthrough')

transfomed_X_train = pd.DataFrame(col_transformer.fit_transform(X_train_baseline))


In [15]:
transfomed_X_train


,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,-0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.651187,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.528816,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2.646256,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37910,1.942935,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
37911,1.880808,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
37912,2.213377,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37913,-0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [16]:
scaled_knn = KNeighborsRegressor(n_neighbors=10).fit(transfomed_X_train,
                                                       y_train_baseline)


### Breath of the wild Test

In [17]:
predict_baseline_model(12552, scaled_knn, final_data, transfomed_X_train)


,title,release_date,plays,playing,backlogs,wishlist,developers,avg_review,genres,platforms,...,ratings_one_five,ratings_two_zero,ratings_two_five,ratings_three_zero,ratings_three_five,ratings_four_zero,ratings_four_five,ratings_five_zero,url,game_id
12552,The Legend of Zelda: Breath of the Wild,1488499200000,44000,3500,6900,3600,"[Nintendo EPD Production Group No. 3, Nintendo]",4.4,"[Adventure, Puzzle, RPG]","[Wii U, Nintendo Switch]",...,131,367,563,1257,2163,5236,6908,13704,/games/the-legend-of-zelda-breath-of-the-wild/,7346
27827,The Legend of Zelda: The Wind Waker,1039737600000,15000,337,4000,1900,"[Nintendo EAD Software Development Group No.3,...",4.2,"[Adventure, Puzzle, RPG]",[Nintendo GameCube],...,24,92,118,439,942,2282,2144,2513,/games/the-legend-of-zelda-the-wind-waker/,1033
17807,The Legend of Zelda: A Link Between Worlds,1385078400000,13000,220,3600,1600,"[Nintendo, Nintendo EAD Software Development G...",4.1,"[Adventure, Puzzle, RPG]",[Nintendo 3DS],...,21,70,134,527,1109,2598,1940,1391,/games/the-legend-of-zelda-a-link-between-worlds/,2909
26557,The Legend of Zelda: The Minish Cap,1099526400000,12000,406,3800,1200,"[Nintendo, Capcom Production Studio 1]",3.9,"[Adventure, Puzzle, RPG]","[Wii U, Nintendo 3DS, Game Boy Advance]",...,17,83,170,694,1494,2317,1120,813,/games/the-legend-of-zelda-the-minish-cap/,1035
19153,Dishonored,1349740800000,20000,382,4900,1200,"[Arkane Studios, Bethesda Softworks]",4.0,"[Adventure, Puzzle, RPG]","[Windows PC, Xbox 360, PlayStation 3]",...,50,208,379,1135,2131,3753,2218,1877,/games/dishonored/,533
25419,Ōkami,1145491200000,6500,224,4300,2100,"[Clover Studio, Capcom]",4.1,"[Adventure, Puzzle, RPG]","[PlayStation 3, Wii, PlayStation 2]",...,20,61,74,237,372,833,649,923,/games/okami/,1271
24496,The Legend of Zelda: Phantom Hourglass,1182556800000,8900,134,2400,824,"[Nintendo, Nintendo EAD Software Development G...",3.3,"[Adventure, Puzzle, RPG]","[Wii U, Nintendo DS]",...,98,316,507,1184,1138,845,250,185,/games/the-legend-of-zelda-phantom-hourglass/,1037
22016,The Legend of Zelda: Spirit Tracks,1260144000000,6800,112,2300,809,"[Nintendo, Nintendo EAD Software Development G...",3.4,"[Adventure, Puzzle, RPG]","[Wii U, Nintendo DS]",...,76,219,354,816,951,694,274,174,/games/the-legend-of-zelda-spirit-tracks/,1038
29044,The Legend of Zelda: Oracle of Seasons,983232000000,4100,111,2300,618,"[Gradiente, Capcom]",3.7,"[Adventure, Puzzle, RPG]","[Nintendo 3DS, Game Boy Color]",...,11,55,125,356,556,650,245,180,/games/the-legend-of-zelda-oracle-of-seasons/,1032
29043,The Legend of Zelda: Oracle of Ages,983232000000,3700,104,2300,638,"[Gradiente, Capcom Production Studio 1]",3.7,"[Adventure, Puzzle, RPG]","[Nintendo 3DS, Game Boy Color]",...,7,47,109,292,489,633,245,186,/games/the-legend-of-zelda-oracle-of-ages/,1041


### FIFA 21

In [18]:
predict_baseline_model(5919, scaled_knn, final_data, transfomed_X_train)


,title,release_date,plays,playing,backlogs,wishlist,developers,avg_review,genres,platforms,...,ratings_one_five,ratings_two_zero,ratings_two_five,ratings_three_zero,ratings_three_five,ratings_four_zero,ratings_four_five,ratings_five_zero,url,game_id
5919,FIFA 21,1602115200000,1600,39,57,20,"[Electronic Arts, EA Vancouver]",2.5,"[Simulator, Sport]","[Windows PC, PlayStation 4, Xbox One, PlayStat...",...,90,176,211,255,114,70,18,12,/games/fifa-21/,134101
8060,NBA 2K20,1567641600000,1200,11,159,7,"[2K Sports, Visual Concepts]",2.6,"[Simulator, Sport]","[Windows PC, PlayStation 4, iOS, Xbox One, Nin...",...,62,109,136,175,100,75,15,10,/games/nba-2k20/,114285
11731,Everybody's Golf,1503964800000,337,9,95,42,"[Clap Hanz, Sony Interactive Entertainment]",3.7,"[Simulator, Sport]",[PlayStation 4],...,1,5,7,39,45,66,19,16,/games/everybodys-golf--1/,28187
24335,Skate,1189641600000,1100,7,141,78,"[EA Black Box, Electronic Arts]",3.5,"[Simulator, Sport]","[Xbox 360, PlayStation 3, Mobile]",...,6,31,35,128,151,147,37,20,/games/skate/,2585
36336,NBA 2K21,1599091200000,756,23,275,13,"[2K Games, Visual Concepts]",2.4,"[Simulator, Sport]","[Windows PC, PlayStation 4, Xbox One, PlayStat...",...,47,102,86,106,55,35,10,3,/games/nba-2k21/,134599
2187,NBA 2K23,1662595200000,550,81,120,16,"[Visual Concepts, 2K Games]",2.7,"[Simulator, Sport]","[Windows PC, PlayStation 4, Xbox One, PlayStat...",...,25,52,54,108,62,42,5,8,/games/nba-2k23/,207393
14952,WWE 2K16,1445904000000,923,4,55,19,"[2K, Visual Concepts]",3.0,"[Simulator, Sport]","[Windows PC, Xbox 360, PlayStation 4, PlayStat...",...,31,51,94,194,119,67,20,13,/games/wwe-2k16/,11057
6189,PGA Tour 2K21,1597881600000,212,10,181,8,"[HB Studios, 2K]",2.9,"[Simulator, Sport]","[Windows PC, PlayStation 4, Xbox One, Nintendo...",...,5,13,34,32,34,16,3,1,/games/pga-tour-2k21/,133939
27730,Dead or Alive Xtreme Beach Volleyball,1043193600000,246,2,78,54,"[Team NINJA, Tecmo]",3.0,"[Simulator, Sport]",[Xbox],...,14,16,18,22,18,18,4,20,/games/dead-or-alive-xtreme-beach-volleyball/,1390
7580,Football Manager 2020,1574121600000,425,14,205,5,[Sports Interactive],3.7,"[Simulator, Sport]","[Windows PC, Google Stadia]",...,6,5,20,40,68,106,27,24,/games/football-manager-2020/,122080


### Omori test

In [19]:
predict_baseline_model(36255, scaled_knn, final_data, transfomed_X_train)


,title,release_date,plays,playing,backlogs,wishlist,developers,avg_review,genres,platforms,...,ratings_one_five,ratings_two_zero,ratings_two_five,ratings_three_zero,ratings_three_five,ratings_four_zero,ratings_four_five,ratings_five_zero,url,game_id
36255,Omori,1608854400000,11000,1400,6700,5200,"[OMOCAT, PLAYISM]",4.1,"[Adventure, Indie, RPG, Turn Based Strategy]","[Windows PC, Mac, PlayStation 4, Nintendo 3DS,...",...,104,191,254,491,766,1552,1587,3009,/games/omori/,26673
15176,Undertale,1442275200000,39000,907,7400,2600,"[tobyfox, 8-4]",4.3,"[Adventure, Indie, RPG, Turn Based Strategy]","[Windows PC, Mac, Linux, PlayStation 4, Xbox O...",...,175,400,540,1492,2521,5560,5488,9709,/games/undertale/,12517
813,Sea of Stars,1693180800000,1200,960,1900,2800,[Sabotage],3.9,"[Adventure, Indie, RPG, Turn Based Strategy]","[Windows PC, PlayStation 4, Xbox One, PlayStat...",...,5,29,47,75,128,239,177,172,/games/sea-of-stars/,131890
7563,Bug Fables: The Everlasting Sapling,1574294400000,1500,179,1600,704,"[DANGEN Entertainment, Moonsprout Games]",4.1,"[Adventure, Indie, RPG, Turn Based Strategy]","[Windows PC, PlayStation 4, Xbox One, Nintendo...",...,5,18,20,74,95,222,220,208,/games/bug-fables-the-everlasting-sapling/,119190
5923,Ikenfell,1602115200000,299,21,315,226,"[Chevy Ray Johnston, Humble Games]",3.7,"[Adventure, Indie, RPG, Turn Based Strategy]","[Windows PC, Mac, PlayStation 4, Xbox One, Nin...",...,2,7,12,25,39,56,27,24,/games/ikenfell/,23873
36514,Grimm's Hollow,1572480000000,647,19,384,109,[ghosthunter],3.6,"[Adventure, Indie, RPG, Turn Based Strategy]",[Windows PC],...,2,8,31,103,125,119,40,34,/games/grimms-hollow/,125110
7284,Temtem,1579564800000,753,98,396,316,"[CremaGames, Humble Bundle]",3.0,"[Adventure, Indie, RPG, Turn Based Strategy]","[Windows PC, PlayStation 4, Xbox One, PlayStat...",...,16,58,60,110,78,71,21,10,/games/temtem/,100357
34804,Wasteland,568080000000,134,2,210,64,"[Electronic Arts, Interplay Productions]",2.9,"[Adventure, Indie, RPG, Turn Based Strategy]","[Windows PC, Mac, Linux, PC DOS, Apple II, Com...",...,5,13,10,20,10,8,3,3,/games/wasteland/,3100
3255,Pokémon Legends: Arceus,1643328000000,14000,1800,3900,2700,"[Nintendo, Game Freak]",3.7,"[Adventure, RPG, Turn Based Strategy]",[Nintendo Switch],...,121,277,492,1195,1909,2627,1390,779,/games/pokemon-legends-arceus/,144054
7306,Yakuza: Like a Dragon,1579132800000,8600,1100,6900,3600,"[Sega, Ryū Ga Gotoku Studios]",4.3,"[Adventure, RPG, Turn Based Strategy]","[Windows PC, PlayStation 4, Xbox One, PlayStat...",...,45,103,129,315,515,1308,1635,2077,/games/yakuza-like-a-dragon/,36550


#### Breath of the wild
This games that are not neccesarily similar, but are simialar based on title and them having the same genre.

#### FIFA 21
Recomendations are much improved here!!! This is because it now recommends diffferent sport games.

#### Omori
Omori recommendations are decent, however it is seen as a dark game, but Sea of Stars and Bug Fables are far from that. 


# Conclusion
Recommendations are much improved, however there are nuances that we are missing


## Functions for packages

In [ ]:
def multilable_encoding(data, column):
    """
    This ohe encodes categories that are stored in lists
    """
    # Initilizing encoder and transforming the data
    mlb_genre = MultiLabelBinarizer()
    transformed_genre = mlb_genre.fit_transform(data[column])

    # Labeling newly created features
    genre_ohe_colums = pd.DataFrame(transformed_genre, columns=mlb_genre.classes_)
    # Concatinating the data
    data.drop('genres', axis=1, inplace=True)
    scaled_data = pd.concat((data, genre_ohe_colums), axis=1)

    return scaled_data


In [ ]:
def yeo_johnson_scaling(X_train, column):
    """
    This scales skewed data, and must be fed X_train and not the entire dataset
    Otherwise you looks visibility on your y, or just make it harder to find
    """
    num_transformer_yeo = PowerTransformer(method='yeo-johnson', standardize=False)

    col_transformer = ColumnTransformer([('num_transformer', num_transformer_yeo,
                                    [column])],
                                    remainder='passthrough')

    transfomed_X_train = pd.DataFrame(col_transformer.fit_transform(X_train))

    return transfomed_X_train
